# Date of the transfer: how many days after the beginning of the open window?

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib
import matplotlib.pyplot as plt

In [2]:
d_tf  = pd.read_csv('data_15.csv', encoding = "utf8")
d_tf = d_tf.drop(['Unnamed: 0'], axis=1)
dopen = pd.read_excel('dateOpening.xlsx', encoding="utf8")

dopen.rename(columns={'Coutnry':'country_name_buying'}, 
                 inplace=True)

d_tf.country_name_selling = d_tf.country_name_selling.replace('Netherland','Netherlands')
d_tf.country_name_buying = d_tf.country_name_buying.replace('Netherland','Netherlands')

d_tf2 = d_tf.join(dopen.set_index('country_name_buying'), on='country_name_buying')

# Add column daysFromOpenBuying
We have different days of opening of the transfer window.


In [3]:
d_tf2['date'] = pd.to_datetime(d_tf2['date'], format="%Y-%m-%d")

In [4]:
d_tf2['s2016'] = pd.to_datetime(d_tf2['s2016'], format="%Y-%m-%d")
d_tf2['s2017'] = pd.to_datetime(d_tf2['s2017'], format="%Y-%m-%d")
d_tf2['s2018'] = pd.to_datetime(d_tf2['s2018'], format="%Y-%m-%d")
d_tf2['w2017'] = pd.to_datetime(d_tf2['w2017'], format="%Y-%m-%d")
d_tf2['w2018'] = pd.to_datetime(d_tf2['w2018'], format="%Y-%m-%d")
d_tf2['w2019'] = pd.to_datetime(d_tf2['w2019'], format="%Y-%m-%d")


In [5]:
d_tf2['days_from_openBuying'] = np.nan

def daysFromOpenwindow(df):
    if df['year'] == 2016 and df['wintersummer']=='s':
        return df['date'] - df['s2016']
    elif df['year'] == 2017 and df['wintersummer']=='s':
        return df['date'] - df['s2017']
    elif df['year'] == 2018 and df['wintersummer']=='s':
        return df['date'] - df['s2018']
    elif df['year'] == 2017 and df['wintersummer']=='w':
        return df['date'] - df['w2017']
    elif df['year'] == 2018 and df['wintersummer']=='w':
        return df['date'] - df['w2018']
    elif df['year'] == 2019 and df['wintersummer']=='w':
        return df['date'] - df['w2019']
    else:
        return np.nan

d_tf2['days_from_openBuying'] = d_tf2.apply(daysFromOpenwindow, axis = 1)

In [6]:
# drop columns with the rankings of all years
d_tf2 = d_tf2.drop(['s2016','s2017','s2018','w2017','w2018','w2019'], axis=1)

In [7]:
def takeOnlyNumberDays(x):
    x=str(x).split(' ')[0]
    return x

d_tf2['days_from_openBuying'] = d_tf2.days_from_openBuying.apply(takeOnlyNumberDays)

In [8]:
d_tf2.days_from_openBuying = d_tf2.days_from_openBuying.apply(int)

In [9]:
sum(d_tf2['days_from_openBuying']<0)

109

In [10]:
#check NA
#d_tf2.isna().sum()

# European rank per nation year before

In [11]:
dr = pd.read_excel('UEFAperCountry.xlsx', encoding="utf8")

dr.rename(columns={'Country':'country_name_selling'}, 
                 inplace=True)

d_tf3 = d_tf2.join(dr.set_index('country_name_selling'), on='country_name_selling')

In [12]:
d_tf3['europeanPerf_country_from'] = np.nan
d_tf3['europeanPerf_country_to'] = np.nan

def perfEurCountryYearBefore(df):
        if df['season']=='16/17':
            return df['e2016']
        elif df['season']=='17/18':
            return df['e2017']
        elif df['season']=='18/19':
            return df['e2018']
        else:
            return np.nan

d_tf3['europeanPerf_country_from'] = d_tf3.apply(perfEurCountryYearBefore, axis = 1)
d_tf3 = d_tf3.drop(columns=['e2016','e2017','e2018'], axis = 1)

dr.rename(columns={'country_name_selling':'country_name_buying'}, 
                 inplace=True)
d_tf3 = d_tf3.join(dr.set_index('country_name_buying'), on='country_name_buying')

d_tf3['europeanPerf_country_to'] = d_tf3.apply(perfEurCountryYearBefore, axis = 1)


In [13]:
d_tf3 = d_tf3.drop(columns=['e2016','e2017','e2018'], axis = 1)

In [14]:
d_tf3.to_csv('data_16.csv', sep=',', header=True)